*Загрузим необходимые пакеты*

In [13]:
%pip install -q torch transformers datasets accelerate peft bitsandbytes trl wandb transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


# *Level 1*

*Установим модель и датасет*

In [7]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
checkpoint = "HuggingFaceTB/SmolLM2-135M-Instruct"

device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint).to(device)

messages = [{"role": "user", "content": "What is gravity?"}]
input_text=tokenizer.apply_chat_template(messages, tokenize=False)
print(input_text)
inputs = tokenizer.encode(input_text, return_tensors="pt").to(device)
outputs = model.generate(inputs, max_new_tokens=50, temperature=0.2, top_p=0.9, do_sample=True)

print(tokenizer.decode(outputs[0]))

<|im_start|>system
You are a helpful AI assistant named SmolLM, trained by Hugging Face<|im_end|>
<|im_start|>user
What is gravity?<|im_end|>

<|im_start|>system
You are a helpful AI assistant named SmolLM, trained by Hugging Face<|im_end|>
<|im_start|>user
What is gravity?<|im_end|>
<|im_start|>assistant
Gravity is a fundamental force of nature that attracts objects with mass towards each other. It is a result of the interaction between mass, energy, and space itself. In the context of our solar system, gravity is responsible for the


In [12]:
from datasets import load_dataset

ds = load_dataset("juyoungml/HelpSteer2-binarized")

train_data = ds["train"]
val_data = ds["validation"]

print(f"Train size: {len(train_data)}, Val size: {len(val_data)}")
print(train_data[0])

Generating validation split: 100%|██████████| 373/373 [00:00<00:00, 111223.90 examples/s]

Train size: 7224, Val size: 373
{'prompt': '\n\nQuestion: \nIf I want to pick one signature appetizer to represent each Asian country, what do you suggest? and why？\n\nSingapore \nMalaysia\nIndonesia \nCambodia \nThailand\nThe Philippines \nVietnam\nGuam\nHawaii \nKorea \nJapan \nTaiwan \nMacau \nHong Kong \nChina\nIndia \nTurkey', 'chosen': 'Here are some suggestions for a signature appetizer from each Asian country:\n\nSingapore: Chili Crab Buns - a delicious combination of sweet and savory flavors, with a spicy kick from the chili crab sauce.\n\nMalaysia: Satay - grilled skewers of meat (usually chicken or beef) served with a delicious peanut sauce.\n\nIndonesia: Gado-Gado - a delicious salad made with boiled vegetables, served with a peanut sauce dressing.\n\nCambodia: Amok - a traditional Khmer curry, served in a coconut shell and made with fish, lemongrass, and other herbs and spices.\n\nThailand: Tom Yum Goong - a spicy and sour soup made with shrimp, lemongrass, and other herbs

*Перейдем к обучению reward модели*